In [110]:
import pandas as pd
 
#this assumes one json item per line in json file
df=pd.read_csv("/content/teste.csv", sep=';')

In [111]:
df.dtypes

titulo         object
comentario     object
classe        float64
dtype: object

In [112]:
#number of rows (datapoints)
len(df)

100

In [113]:
df.sample(10)

,titulo,comentario,classe
59,Não compre este jogo! Pelo menos não agora!,Eu comprei este jogo para o meu PS4 padrão e e...,0.0
27,Recomendo,Gostei do jogo,1.0
50,NÃO COMPREM,O jogo está completamente injogável no PS4. Gr...,0.0
10,incrível,game chagou super rápido e a CD project é incr...,1.0
97,Muito ruim,"Horrível. Bugado! Impossível de jogar, trava d...",0.0
34,Jogo Cyberpaunk,Muito bom. Chegou 15 dias antes do previsto. A...,1.0
21,Ótimo,"Ótimo produto, entrega rápida, infelizmente o ...",1.0
44,Ótimo,"Ótimo jogo ,. Ale a pena",1.0
89,O produto não é funcional,A desenvolvedora do jogo lançou um produto ina...,0.0
67,"Jogo esta quebrado, mal otimizado.","Não comprem esse jogo para atual geração, ele ...",0.0


In [114]:
#titulo e comentário
df['titulo_comentario'] = df['titulo'] + ' ' + df['comentario']

df.sample(10)

,titulo,comentario,classe,titulo_comentario
13,Muito Bom!,"Sinceramente, o jogo melhorou bastante depois ...",1.0,"Muito Bom! Sinceramente, o jogo melhorou basta..."
55,Impossível de ser jogado,"FPS baixo, definição baixa, o jogo praticament...",0.0,"Impossível de ser jogado FPS baixo, definição ..."
93,Jogo tosco,Não consigo nem terminar o jogo de tanto bugs ...,0.0,Jogo tosco Não consigo nem terminar o jogo de ...
11,Que é um super jogo! Vale cada centavo :),Melhor jogo!!! Entregue bem antes da data prev...,1.0,Que é um super jogo! Vale cada centavo :) Melh...
16,Gostei,Estou jogando no PS5 e tendo uma ótima experiê...,1.0,Gostei Estou jogando no PS5 e tendo uma ótima ...
51,Gráfico de Ps2 no Ps5,Jogo cheio de bugs e gráficos de PS2 em divers...,0.0,Gráfico de Ps2 no Ps5 Jogo cheio de bugs e grá...
61,COMPLETAMENTE CHEIO DE BUGS E DEFEITOS,"JOGO É HORRIVEL, COMPLETAMENTE CHEIO DE PROBLE...",0.0,COMPLETAMENTE CHEIO DE BUGS E DEFEITOS JOGO É ...
33,Satisfeito.,Pré-venda Amazon sempre satisfatória.,1.0,Satisfeito. Pré-venda Amazon sempre satisfatória.
64,Injogavel no momento,"O jogo tem muito potencial, mas infelizmente n...",0.0,Injogavel no momento O jogo tem muito potencia...
28,Qualidade da entrega,Chegou tudo bem perfeitinho. Muito obrigado pe...,1.0,Qualidade da entrega Chegou tudo bem perfeitin...


In [115]:
def _reciprocal_rank(true_labels: list, machine_preds: list):
    """Compute the reciprocal rank at cutoff k"""
    
    # add index to list only if machine predicted label exists in true labels
    tp_pos_list = [(idx + 1) for idx, r in enumerate(machine_preds) if r in true_labels]

    rr = 0
    if len(tp_pos_list) > 0:
        # for RR we need position of first correct item
        first_pos_list = tp_pos_list[0]
        
        # rr = 1/rank
        rr = 1 / float(first_pos_list)

    return rr

def compute_mrr_at_k(items:list):
    """Compute the MRR (average RR) at cutoff k"""
    rr_total = 0
    
    for item in items:   
        rr_at_k = _reciprocal_rank(item[0],item[1])
        rr_total = rr_total + rr_at_k
        mrr = rr_total / 1/float(len(items))

    return mrr

def collect_preds(Y_test,Y_preds):
    """Collect all predictions and ground truth"""
    
    pred_gold_list=[[[Y_test[idx]],pred] for idx,pred in enumerate(Y_preds)]
    return pred_gold_list
             
def compute_accuracy(eval_items:list):
    correct=0
    total=0
    
    for item in eval_items:
        true_pred=item[0]
        machine_pred=set(item[1])
        
        for cat in true_pred:
            if cat in machine_pred:
                correct+=1
                break
    
    
    accuracy=correct/float(len(eval_items))
    return accuracy

In [151]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import numpy as np
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def extract_features(df,field,training_data,testing_data,type="binary"):
    """Extract features using different methods"""
    
    logging.info("Extracting features and creating vocabulary...")
    
    if "binary" in type:
        
        # BINARY FEATURE REPRESENTATION
        cv= CountVectorizer(binary=True, max_df=0.95)
        cv.fit_transform(training_data[field].values)
        
        train_feature_set=cv.transform(training_data[field].values)
        test_feature_set=cv.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,cv
  
    elif "counts" in type:
        
        # COUNT BASED FEATURE REPRESENTATION
        cv= CountVectorizer(binary=False, max_df=0.95)
        cv.fit_transform(training_data[field].values)
        
        train_feature_set=cv.transform(training_data[field].values)
        test_feature_set=cv.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,cv
    
    else:    
        print('Using TF-IDF')
        # TF-IDF BASED FEATURE REPRESENTATION
        tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95)
        tfidf_vectorizer.fit_transform(training_data[field].values)
        
        train_feature_set=tfidf_vectorizer.transform(training_data[field].values)
        test_feature_set=tfidf_vectorizer.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,tfidf_vectorizer

def get_top_k_predictions(model,X_test,k=1):
    
    # get probabilities instead of predicted labels, since we want to collect top 3
    probs = model.predict_proba(X_test)

    # GET TOP K PREDICTIONS BY PROB - note these are just index
    best_n = np.argsort(probs, axis=1)[:,-k:]
    
    # GET CATEGORY OF PREDICTIONS
    preds=[[model.classes_[predicted_cat] for predicted_cat in prediction] for prediction in best_n]
    
    preds=[ item[::-1] for item in preds]
    
    return preds
   
    
def train_model(df,field="text_desc",feature_rep="binary",top_k=1):
    
    print("Starting model training...")
    
    # GET A TRAIN TEST SPLIT (set seed for consistent results)
    training_data, testing_data = train_test_split(df,random_state = 2000,)

    # GET LABELS
    Y_train=training_data['classe'].values
    Y_test=testing_data['classe'].values
     
    # GET FEATURES
    X_train,X_test,feature_transformer=extract_features(df,field,training_data,testing_data,type=feature_rep)

    # INIT LOGISTIC REGRESSION CLASSIFIER
    print("Training a Logistic Regression Model...")
    #scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
    scikit_log_reg = LogisticRegression()
    model=scikit_log_reg.fit(X_train,Y_train)
    

    # GET TOP K PREDICTIONS
    preds=get_top_k_predictions(model,X_test,top_k)
    
    # GET PREDICTED VALUES AND GROUND TRUTH INTO A LIST OF LISTS - for ease of evaluation
    eval_items=collect_preds(Y_test,preds)
    
    # GET EVALUATION NUMBERS ON TEST SET -- HOW DID WE DO?
    print("Starting evaluation...")
    accuracy=compute_accuracy(eval_items)
    mrr_at_k=compute_mrr_at_k(eval_items)
    
    print("Done training and evaluation.")
    
    return model,feature_transformer,accuracy,mrr_at_k
    #return model,feature_transformer

In [154]:
field='titulo_comentario'
feature_rep='counts'

model,transformer,accuracy,mrr_at_k=train_model(df,field=field,feature_rep=feature_rep,top_k=1)
print("\nAccuracy={0}; MRR={1}".format(accuracy,mrr_at_k))
#model,transformer=train_model(df,field=field,feature_rep=feature_rep)

2021-02-04 17:48:55,889 : INFO : Extracting features and creating vocabulary...


Starting model training...
Training a Logistic Regression Model...
Starting evaluation...
Done training and evaluation.

Accuracy=0.84; MRR=0.84


In [159]:
test_features=transformer.transform(["O jogo está péssimo"])
preds = get_top_k_predictions(model,test_features,1)

if preds[0][0] == 0.0:
  print('Comentário ruim')
else:
  print('Comentário bom')

Comentário ruim
